In [1]:
import vamb

import numpy as np
import torch as torch
import torch.nn as nn
from torch.optim import Adam as Adam
from torch.utils.data import DataLoader as DataLoader
from torch.utils.data import TensorDataset

import wandb

from collections import namedtuple

import os

import json

import sys

import pandas as pd

torch.manual_seed(0)

# Load In Data

In [2]:
BASE_DIR = os.getcwd()

EXAMPLE_FASTA_FILE = '2021.01.26_15.46.45_sample_0'

In [3]:
vamb_inputs_base = os.path.join(BASE_DIR,'example_input_data/new_simulations/camisim_outputs/vamb_inputs')

contignames = vamb.vambtools.read_npz(os.path.join(vamb_inputs_base, 'contignames.npz'))

lengths = vamb.vambtools.read_npz(os.path.join(vamb_inputs_base, 'lengths.npz'))

tnfs = vamb.vambtools.read_npz(os.path.join(vamb_inputs_base, 'tnfs.npz'))
    
rpkms = vamb.vambtools.read_npz(os.path.join(vamb_inputs_base, 'rpkms.npz'))

# Adapt Through DataLoader

In [4]:
depthssum = rpkms.sum(axis=1)
mask = tnfs.sum(axis=1) != 0
mask &= depthssum != 0
depthssum = depthssum[mask]

rpkm = rpkms[mask].astype(np.float32, copy=False)
tnf = tnfs[mask].astype(np.float32, copy=False)

## lkj
def calculate_z_score(array):
    array_mean = array.mean(axis=0)
    array_std = array.std(axis=0)

    shape = np.copy(array.shape)
    shape[0] = 1
    shape = tuple(shape)

    array_mean.shape = shape
    array_mean.shape = shape

    array = (array - array_mean) / array_std
    
    return(array)
    
rpkm = calculate_z_score(rpkm)
tnf = calculate_z_score(tnf)
depthstensor = torch.from_numpy(rpkm)
tnftensor = torch.from_numpy(tnf)

n_workers = 4

dataset = TensorDataset(depthstensor, tnftensor)
dataloader = DataLoader(dataset=dataset, batch_size=256, drop_last=True,
                             shuffle=True, num_workers=n_workers, pin_memory=True)



ncontigs, nsamples = dataset.tensors[0].shape

# VAE Model

In [5]:
class DISENTANGLED_BETA_VAE(torch.nn.Module):
    def __init__(self, nsamples, config):
        super(DISENTANGLED_BETA_VAE, self).__init__()
  
        # SET UP AND CONFIGURE THE MODEL
        self.ntnf = tnfs.shape[1]
        
        self.nlatent = config.nlatent
        self.dropout = config.dropout
        self.learning_rate = config.learning_rate
        self.alpha = config.alpha
        self.beta = config.beta
        self.nepochs = config.nepochs
        
        nhiddens = [512, 512]
        
        self.nsamples = nsamples
        self.cuda_on = False
        
        if self.cuda_on:
            self.cuda()

        self.encoderlayers = torch.nn.ModuleList()
        self.encodernorms = torch.nn.ModuleList()
        self.decoderlayers = torch.nn.ModuleList()
        self.decodernorms = torch.nn.ModuleList()


        # ENCODER LAYERS
        self.encoderlayers.append( torch.nn.Linear((self.nsamples + self.ntnf), 512) )
        self.encodernorms.append( torch.nn.BatchNorm1d(512) )

        self.encoderlayers.append( torch.nn.Linear(512, 512) )
        self.encodernorms.append( torch.nn.BatchNorm1d(512) )


        # LATENT LAYERS
        self.mu = torch.nn.Linear(512, self.nlatent)
        self.logsigma = torch.nn.Linear(512, self.nlatent)


        # DECODER LAYRS
        self.decoderlayers.append(torch.nn.Linear(self.nlatent, 512))
        self.decodernorms.append(torch.nn.BatchNorm1d(512))

        self.decoderlayers.append(torch.nn.Linear(512, 512))
        self.decodernorms.append(torch.nn.BatchNorm1d(512))


        # RECONSTRUCTION LAYER
        self.outputlayer = torch.nn.Linear(512, (self.nsamples + self.ntnf) )


        # ACTIVATIONS
        self.relu = torch.nn.LeakyReLU()
        self.softplus = torch.nn.Softplus()
        self.dropoutlayer = torch.nn.Dropout(p=self.dropout)

        
    ###
    # ENCODE NEW CONTIGS TO LATENT SPACE
    ###
    def encode(self, data_loader):
        self.eval()

        new_data_loader = DataLoader(dataset=data_loader.dataset,
                                      batch_size=data_loader.batch_size,
                                      shuffle=False,
                                      drop_last=False,
                                      num_workers=4,
                                      pin_memory=data_loader.pin_memory)

        depths_array, tnf_array = data_loader.dataset.tensors
        length = len(depths_array)

        latent = np.empty((length, self.nlatent), dtype=np.float32)

        row = 0
        with torch.no_grad():
            for depths, tnf in new_data_loader:
                if self.cuda_on:
                    depths = depths.cuda()
                    tnf = tnf.cuda()

                # Evaluate
                out_depths, out_tnf, mu, logsigma = self(depths, tnf)

                if self.cuda_on:
                    mu = mu.cpu()

                latent[row: row + len(mu)] = mu
                row += len(mu)

        assert row == length
        return latent
    
    ###
    # SPECIFIC ENCODING AND DECODING FUNCTIONS
    ###
    # REPARAMATERIZE
    def reparameterize(self, mu, logsigma):
        epsilon = torch.randn(mu.size(0), mu.size(1))

        if self.cuda_on:
            epsilon = epsilon.cuda()

        epsilon.requires_grad = True

        # See comment above regarding softplus
        latent = mu + epsilon * torch.exp(logsigma/2)

        return latent
    
    
    # ENCODE CONTIGS
    def encode_contigs(self, tensor):
        tensors = list()

        # Hidden layers
        for encoderlayer, encodernorm in zip(self.encoderlayers, self.encodernorms):
            tensor = encodernorm(self.dropoutlayer(self.relu(encoderlayer(tensor))))
            tensors.append(tensor)

        # Latent layers
        mu = self.mu(tensor)
        logsigma = self.softplus(self.logsigma(tensor))

        return mu, logsigma
    
    
    # DECODE CONTIGS
    def decode_contigs(self, tensor):
        tensors = list()

        for decoderlayer, decodernorm in zip(self.decoderlayers, self.decodernorms):
            tensor = decodernorm(self.dropoutlayer(self.relu(decoderlayer(tensor))))
            tensors.append(tensor)

        reconstruction = self.outputlayer(tensor)

        # Decompose reconstruction to depths and tnf signal
        depths_out = reconstruction.narrow(1, 0, self.nsamples)
        tnf_out = reconstruction.narrow(1, self.nsamples, tnfs.shape[1])

        return depths_out, tnf_out
    
    
    ###
    # LOSS CALCULATION
    ###
    # CALCULATE LOSS
    def calculate_loss(self, depths_in, depths_out, tnf_in, tnf_out, mu, logsigma):
        depths_loss = (depths_out - depths_in).pow(2).sum(dim=1).mean()
        depths_loss_weight = 1 - self.alpha # alpha

        tnfs_loss = (tnf_out - tnf_in).pow(2).sum(dim=1).mean()
        tnfs_loss_weight = (self.alpha / self.ntnf)
        
        kld = -0.5 * (1 + logsigma - mu.pow(2) - logsigma.exp()).sum(dim=1).mean()

        
        # BETA PARAMETER HERE
        loss = (depths_loss * depths_loss_weight) + (tnfs_loss * tnfs_loss_weight) + (self.beta * kld)

        return loss, depths_loss, tnfs_loss, kld
    

    ###
    # TRAINING FUNCTIONS
    ###
    # FORWARD
    def forward(self, depths, tnf):
        tensor = torch.cat((depths, tnf), 1)
        mu, logsigma = self.encode_contigs(tensor)
        latent = self.reparameterize(mu, logsigma)
        depths_out, tnf_out = self.decode_contigs(latent)

        return depths_out, tnf_out, mu, logsigma   
        
     
    
    # TRAIN SPECIFIC EPOCH
    def trainepoch(self, data_loader, epoch, optimizer, batchsteps):
        self.train()

        epoch_loss, epoch_kldloss, epoch_tnfs_loss, epoch_depths_loss = 0, 0, 0, 0

        if epoch in batchsteps:
            data_loader = DataLoader(dataset=data_loader.dataset,
                                      batch_size=data_loader.batch_size * 2,
                                      shuffle=True,
                                      drop_last=True,
                                      num_workers=data_loader.num_workers,
                                      pin_memory=data_loader.pin_memory)

        for depths_in, tnf_in in data_loader:
            depths_in.requires_grad = True
            tnf_in.requires_grad = True

            # CUDA ENABLING
            if self.cuda_on:
                depths_in = depths_in.cuda()
                tnf_in = tnf_in.cuda()

            optimizer.zero_grad()

            depths_out, tnf_out, mu, logsigma = self(depths_in, tnf_in)

            loss, depths_loss, tnfs_loss, kld = self.calc_loss(depths_in, depths_out, tnf_in,
                                                  tnf_out, mu, logsigma)

            loss.backward()
            optimizer.step()

            epoch_loss = epoch_loss + loss.data.item()
            epoch_kldloss = epoch_kldloss + kld.data.item()
            epoch_tnfs_loss = epoch_tnfs_loss + tnfs_loss.data.item()
            epoch_depths_loss = epoch_depths_loss + depths_loss.data.item()

        print('\tEpoch: {}\tLoss: {:.6f}\tCE: {:.7f}\tSSE: {:.6f}\tKLD: {:.4f}\tBatchsize: {}'.format(
              epoch + 1,
              epoch_loss / len(data_loader),
              epoch_depths_loss / len(data_loader),
              epoch_tnfs_loss / len(data_loader),
              epoch_kldloss / len(data_loader),
              data_loader.batch_size,
              ))
        wandb.log({
            "epoch": (epoch+1), 
            "Loss": epoch_loss / len(data_loader),
            "DepthsLoss": epoch_depths_loss / len(data_loader),
            "TNFsLoss": epoch_tnfs_loss / len(data_loader),
            "KLDLoss": epoch_kldloss / len(data_loader),
            "Batchsize": data_loader.batch_size
        })

        return data_loader
    
    
    
    # TRAIN MODEL    
    def trainmodel(self, dataloader, batchsteps=[25, 75, 150, 300], modelfile=None):
        
        batchsteps_set = set()
        
        ncontigs, nsamples = dataloader.dataset.tensors[0].shape
        optimizer = Adam(self.parameters(), lr=self.learning_rate)
        
        
        # TRAIN EPOCH
        for epoch in range(self.nepochs):
            dataloader = self.trainepoch(dataloader, epoch, optimizer, batchsteps_set)


In [7]:
wandb_on = True
sweep_on = True


if wandb_on:
    if sweep_on:
        sweep_config = {
        'method': 'bayes',
        'early_terminate': {
          'type': 'hyperband',
          'max_iter': 27,
          's': 2
        },
        'metric': {
          'name': 'loss',
          'goal': 'minimize'   
            },
            'parameters': {
                'nepochs': {
                    'values': [120]
                },
                'dropout': {
                    'values': [0.2, 0.4, 0.6]
                },
                'learning_rate': {
                    'values': [1e-2, 1e-3, 1e-4, 1e-5]
                },
                'alpha': {
                    'values': [0.15, 0.25]
                },
                'beta': {
                    'values': [1, 200, 400, 800]
                },
                'nlatent': {
                    'values': [32, 64]
                }
            }
        }
        
        
        def train():
            config_defaults = {
                'nepochs': 5,
                'dropout': 0.2,
                'learning_rate': 1e-3,
                'alpha': 0.15,
                'beta': 200,
                'nlatent': 32
            }
            
            wandb.init(project='cs_230_vae', entity='pmccaffrey6', config=config_defaults)
            
            config = wandb.config
            beta_vae = DISENTANGLED_BETA_VAE(nsamples=rpkms.shape[1], config=config)
            wandb.watch(beta_vae)
            beta_vae.trainmodel(dataloader, batchsteps=None)
            print('rundir:', wandb.run.dir)
            torch.save(beta_vae.state_dict(), os.path.join(wandb.run.dir, "model.h5"))
            

        sweep_id = wandb.sweep(sweep_config, entity="pmccaffrey6", project="cs_230_vae")
        wandb.agent(sweep_id, function=train)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: vtxaskgw
Sweep URL: https://wandb.ai/pmccaffrey6/cs_230_vae/sweeps/vtxaskgw


wandb: Agent Starting Run: q5h7addy with config:
wandb: 	alpha: 0.25
wandb: 	beta: 1
wandb: 	dropout: 0.4
wandb: 	learning_rate: 0.001
wandb: 	nepochs: 300
wandb: 	nlatent: 64
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pmccaffrey6 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


	Epoch: 1	Loss: 1.535106	CE: 1.1134370	SSE: 135.248749	KLD: 25.0701	Batchsize: 256
	Epoch: 2	Loss: 1.361063	CE: 0.8750011	SSE: 127.516237	KLD: 27.6230	Batchsize: 256
	Epoch: 3	Loss: 1.155872	CE: 0.6211215	SSE: 121.560579	KLD: 28.8569	Batchsize: 256
	Epoch: 4	Loss: 1.055479	CE: 0.5051520	SSE: 113.664375	KLD: 29.4764	Batchsize: 256
	Epoch: 5	Loss: 1.044187	CE: 0.5395366	SSE: 111.838084	KLD: 27.0535	Batchsize: 256
	Epoch: 6	Loss: 0.917017	CE: 0.4194311	SSE: 106.245908	KLD: 25.9695	Batchsize: 256
	Epoch: 7	Loss: 0.871028	CE: 0.3868858	SSE: 100.416098	KLD: 25.3400	Batchsize: 256
	Epoch: 8	Loss: 0.812877	CE: 0.3316474	SSE: 92.940588	KLD: 25.3201	Batchsize: 256
	Epoch: 9	Loss: 0.779238	CE: 0.3131797	SSE: 90.637419	KLD: 24.3865	Batchsize: 256
	Epoch: 10	Loss: 0.750733	CE: 0.2956007	SSE: 87.233363	KLD: 23.8358	Batchsize: 256
	Epoch: 11	Loss: 0.742194	CE: 0.3132914	SSE: 85.040640	KLD: 22.5312	Batchsize: 256
	Epoch: 12	Loss: 0.712618	CE: 0.2916597	SSE: 81.064717	KLD: 22.1857	Batchsize: 256
	Epoch

epoch,300
loss,0.2801
CELoss,0.1358
SSELoss,54.54661
KLDLoss,5.45462
Batchsize,256
_runtime,152
_timestamp,1615575037
_step,299


...

epoch,300
loss,0.05884
CELoss,0.02108
SSELoss,24.1396
KLDLoss,147.70203
Batchsize,256
_runtime,142
_timestamp,1615585525
_step,299


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▄▃▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
CELoss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
SSELoss,█▆▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
KLDLoss,█▁▁▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Batchsize,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Agent Starting Run: nyd52zhs with config:
wandb: 	alpha: 0.15
wandb: 	beta: 800
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.01
wandb: 	nepochs: 300
wandb: 	nlatent: 32
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


	Epoch: 1	Loss: 135.245642	CE: 6.7190956	SSE: 127.294733	KLD: 3311335.4471	Batchsize: 256
	Epoch: 2	Loss: 1.473420	CE: 1.3893655	SSE: 97.182793	KLD: 3863.8238	Batchsize: 256
	Epoch: 3	Loss: 0.955840	CE: 0.9435926	SSE: 81.874437	KLD: 884.5146	Batchsize: 256
	Epoch: 4	Loss: 0.780711	CE: 0.7704976	SSE: 74.285797	KLD: 450.6725	Batchsize: 256
	Epoch: 5	Loss: 0.665004	CE: 0.6299941	SSE: 69.140248	KLD: 737.7679	Batchsize: 256
	Epoch: 6	Loss: 0.731890	CE: 0.7224483	SSE: 63.277130	KLD: 656.8297	Batchsize: 256
	Epoch: 7	Loss: 0.597013	CE: 0.5674709	SSE: 60.470028	KLD: 680.9369	Batchsize: 256
	Epoch: 8	Loss: 0.617502	CE: 0.5948758	SSE: 58.863238	KLD: 669.0317	Batchsize: 256
	Epoch: 9	Loss: 0.574422	CE: 0.5553800	SSE: 53.731854	KLD: 616.9309	Batchsize: 256
	Epoch: 10	Loss: 0.537008	CE: 0.5219655	SSE: 51.519459	KLD: 468.7157	Batchsize: 256
	Epoch: 11	Loss: 0.525274	CE: 0.5149753	SSE: 48.205990	KLD: 443.9591	Batchsize: 256
	Epoch: 12	Loss: 0.504786	CE: 0.4938672	SSE: 46.465583	KLD: 443.6750	Batchsiz

wandb: Ctrl + C detected. Stopping sweep.


In [13]:
# GET THE BEST MODEL'S PARAMS FROM THE WANDB API
api = wandb.Api()
print()
sweep = api.sweep(f"pmccaffrey6/cs_230_vae/{sweep_id}")

runs = sorted(sweep.runs, key=lambda run: run.summary.get("loss", 0), reverse=True)
loss = runs[0].summary.get("loss", 0)

best_run = runs[0].name

print(f"Best run {best_run} with {loss}% validation accuracy")
best_params_dict = json.loads(runs[0].json_config)
print('best_params_dict:', best_params_dict)

def convert(dictionary):
    return namedtuple('GenericDict', dictionary.keys())(**dictionary)

best_params = convert( {key:best_params_dict[key]['value'] for key in best_params_dict.keys()} )


Best run pleasant-sweep-3 with 1.6544751167297362% validation accuracy
best_params_dict: {'beta': {'value': 1, 'desc': None}, 'alpha': {'value': 0.25, 'desc': None}, 'dropout': {'value': 0.6, 'desc': None}, 'nepochs': {'value': 300, 'desc': None}, 'nlatent': {'value': 32, 'desc': None}, 'learning_rate': {'value': 1e-05, 'desc': None}}


In [15]:
runs[0].file("model.h5").download(replace=True)

<File model.h5 (None) 0.0B>

In [11]:
beta_vae = DISENTANGLED_BETA_VAE(nsamples=rpkms.shape[1], config=best_params)
beta_vae.load_state_dict(torch.load('model.h5'))

latent = beta_vae.encode(dataloader)
print("Latent shape:", latent.shape)

latent_output_path = os.path.join(BASE_DIR, 'example_input_data/new_simulations/camisim_outputs/vamb_outputs/latent_space.npy')
with open(latent_output_path, 'wb') as outfile:
    np.save(outfile, latent)

Latent shape: (1342, 32)


# Leverage Other VAMB Tools for Clustering and Post Steps

In [36]:
contig_mapping_table = pd.read_csv(os.path.join(BASE_DIR, f"example_input_data/new_simulations/camisim_outputs/{EXAMPLE_FASTA_FILE}/contigs/gsa_mapping.tsv"), sep='\t')

contig_mapping_output_path = os.path.join(BASE_DIR, 'example_input_data/new_simulations/camisim_outputs/vamb_outputs/encoding_mapping.tsv')    

contig_mapping_table[contig_mapping_table['#anonymous_contig_id'].isin(contignames)].reset_index().drop('index', axis=1).set_index(
    '#anonymous_contig_id').reindex(contignames).to_csv(contig_mapping_output_path, sep='\t')

In [30]:
filtered_labels = [n for (n,m) in zip(contignames, mask) if m]
cluster_iterator = vamb.cluster.cluster(latent, labels=filtered_labels)
clusters = dict(cluster_iterator)

medoid, contigs = next(iter(clusters.items()))
print('First key:', medoid, '(of type:', type(medoid), ')')
print('Type of values:', type(contigs))
print('First element of value:', next(iter(contigs)), 'of type:', type(next(iter(contigs))))

First key: S0C34345 (of type: <class 'numpy.str_'> )
Type of values: <class 'set'>
First element of value: S0C34345 of type: <class 'numpy.str_'>


In [31]:
def filterclusters(clusters, lengthof):
    filtered_bins = dict()
    for medoid, contigs in clusters.items():
        binsize = sum(lengthof[contig] for contig in contigs)
    
        if binsize >= 200000:
            filtered_bins[medoid] = contigs
    
    return filtered_bins
        
lengthof = dict(zip(contignames, lengths))
filtered_bins = filterclusters(vamb.vambtools.binsplit(clusters, 'C'), lengthof)
print('Number of bins before splitting and filtering:', len(clusters))
print('Number of bins after splitting and filtering:', len(filtered_bins))

Number of bins before splitting and filtering: 1163
Number of bins after splitting and filtering: 3


In [32]:
vamb_outputs_base = os.path.join(BASE_DIR, 'example_input_data/new_simulations/camisim_outputs/vamb_outputs')

if not os.path.exists(vamb_outputs_base):
    os.mkdir(vamb_outputs_base)
    

# This writes a .tsv file with the clusters and corresponding sequences
with open(os.path.join(vamb_outputs_base, 'clusters_dvae.tsv'), 'w') as file:
    vamb.vambtools.write_clusters(file, filtered_bins)

# Only keep contigs in any filtered bin in memory
keptcontigs = set.union(*filtered_bins.values())



# decompress fasta.gz if present
fasta_path = os.path.join(BASE_DIR, f"example_input_data/new_simulations/camisim_outputs/{EXAMPLE_FASTA_FILE}/contigs/anonymous_gsa.fasta.gz")
if os.path.exists(fasta_path) and not os.path.exists(fasta_path.replace('.fasta.gz','.fasta')):
    !gzip -dk $fasta_path


with open(os.path.join(BASE_DIR, f"example_input_data/new_simulations/camisim_outputs/{EXAMPLE_FASTA_FILE}/contigs/anonymous_gsa.fasta"), 'rb') as file:
    fastadict = vamb.vambtools.loadfasta(file, keep=keptcontigs)


bindir = os.path.join(vamb_outputs_base, 'dvae_bins')
if not os.path.exists(bindir):
    os.mkdir(bindir)
vamb.vambtools.write_bins(bindir, filtered_bins, fastadict, maxbins=500)